In [36]:
from langgraph.graph import StateGraph,END,START
from typing import TypedDict
import dotenv
dotenv.load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI


In [37]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
)

In [38]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [39]:
def generate_blog_title_outline(state : BlogState) -> BlogState:
    title = state['title']
    prompt = f"Generate a detailed outline for a blog post titled '{title}'."
    outline = model.invoke(prompt)
    state['outline'] = outline
    return state


In [40]:
def create_blog_post(state : BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    prompt = f"Generate a blog post based on the title '{title}' and outline '{outline}'."
    content = model.invoke(prompt)
    state['content'] = content
    return state

In [41]:
# create nodes
graph = StateGraph(BlogState)

graph.add_node('generate_blog_title_outline', generate_blog_title_outline)
graph.add_node('create_blog_post', create_blog_post)

#  create edges
graph.add_edge(START, 'generate_blog_title_outline')
graph.add_edge('generate_blog_title_outline', 'create_blog_post')
graph.add_edge('create_blog_post', END)

# execute the graph
workflow = graph.compile()

In [ ]:
final_state = workflow.invoke({'title': 'The Future of AI in Everyday Life'})
# print(final_state)
print(final_state['content'])

{'title': 'The Future of AI in Everyday Life', 'outline': AIMessage(content='Of course! Here is a detailed outline for a blog post titled \'The Future of AI in Everyday Life\'. This outline is structured to be comprehensive, engaging, and easy for a writer to follow.\n\n---\n\n### **Blog Post Outline: The Future of AI in Everyday Life**\n\n**Target Audience:** General readers, tech-curious individuals, professionals interested in future trends.\n**Tone:** Informative, speculative, optimistic but grounded, and slightly awe-inspiring.\n**Goal:** To move beyond current AI applications (like Alexa and Netflix) and paint a vivid, practical picture of how AI will be deeply integrated into our daily routines in the next 5-20 years.\n\n---\n\n### **1. Title Options (Choose one or brainstorm from these):**\n\n*   The Future of AI in Everyday Life\n*   Beyond Siri: How AI Will Actually Change Your Daily Routine\n*   Your Life in 2040: A Day Powered by Artificial Intelligence\n*   The Invisible R